In [220]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import unidecode
tqdm.pandas()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore") 

| Kategori                                                               | İçerik Sayısı     | Hedef Değişkenler                                                                                            |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`,`mevzuat_tarihi`, `madde_sayisi`
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`
|
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | `kategori`, `donem` ve `sira_no`                                                                       |
| [Genelge](#kategori-genelge)                                           | 385               | `kategori`, `mevzuat_tarihi` ve varsa `mevzuat_no` yoksa `belge_sayi` 
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `madde_sayisi` ve varsa ` mevzuat_tarihi`
| [Tüzük](#kategori-tüzük)                                               | 82                |  `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum`, `madde_sayisi`       |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum` 
| [Özelge](#kategori-özelge)                                             | 750               | `kategori`, `mevzuat_tarihi`

In [221]:
df=pd.read_csv(r'kanunum-nlp-doc-analysis-dataset.csv')

In [222]:
kanun=df[df.kategori=='Kanun']
khk=df[df.kategori=='Kanun Hükmünde Kararname']
resmi_gazete=df[df.kategori=='Resmi Gazete']
genelge=df[df.kategori=='Genelge']
c_kararname=df[df.kategori=='Genelge']
tüzük=df[df.kategori=='Tüzük']
yönetmelik=df[df.kategori=='Yönetmelik']
teblig=df[df.kategori=='Tebliğ']
ozelge=df[df.kategori=='Özelge']

In [223]:
#def find_date_kanun(text,date_type):
#    if date_type=='rega':
#        match=re.findall(r'\d{2}.\d{2}.\d{4}', text)[0]
#        if len(match)==0:
#            match=pd.to_datetime(re.findall(r'\d{1}.\d{1}.\d{4}', text)[0])
#        match=pd.to_datetime(match)
#        #kanun['data_text'][0].partition("Resmî Gazete Tarihi: ")[2].split('Resmî Gazete Sayısı:')[0]
#        #date = datetime.strptime(match[0], '%d.%m.%Y').date()
#    elif date_type=='mevzuat':
#        match=re.findall(r'\d{2}.\d{2}.\d{4}', text)[1]
#        if len(match)==0:
#            match=pd.to_datetime(re.findall(r'\d{1}.\d{1}.\d{4}', text)[1])
#        match=pd.to_datetime(match)
#        #date = datetime.strptime(match[1], '%d/%m/%Y').date()
#    return(match)

### UTILS

In [257]:
def find_dates(text,date_type,kategori):
    std_txt=unidecode.unidecode(text).lower()
    if kategori=='kanun':
        if date_type=='rega':
            match=std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
        elif date_type=='mevzuat':
            match=std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[1].split('kabul tarihi : ')[1].split('\n')[0]
    elif kategori=='khk':
        if date_type=='rega':
            match=std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
        elif date_type=='mevzuat':
            match=std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[1].split('kararnamenin tarihi : ')[1].split('\n')[0]
    elif kategori=='genelge':
        if date_type=='mevzuat':
            match=std_txt.split("tarih ",1)[1].split('\n')[0]
    return match

### KANUN

In [227]:
kanun['rega_date_val']=kanun['data_text'].progress_apply(lambda x : find_dates(x,'rega','kanun'))
kanun['mevzuat_date_val']=kanun['data_text'].progress_apply(lambda x : find_dates(x,'mevzuat','kanun'))

100%|██████████████████████████████████████████████████████████████████| 514/514 [00:03<00:00, 155.48it/s]


In [228]:
kanun['mevzuat_date_val']=pd.to_datetime(kanun['mevzuat_date_val'],dayfirst=True)
kanun['rega_date_val']=pd.to_datetime(kanun['rega_date_val'],dayfirst=True)

In [229]:
print((kanun['mevzuat_date_val']==kanun['mevzuat_tarihi']).value_counts())
print((kanun['rega_date_val']==kanun['rega_tarihi']).value_counts())

True    514
dtype: int64
True    514
dtype: int64


### KHK

In [230]:
khk['rega_date_val']=khk['data_text'].progress_apply(lambda x : find_dates(x,'rega','khk'))
khk['mevzuat_date_val']=khk['data_text'].progress_apply(lambda x : find_dates(x,'mevzuat','khk'))

100%|████████████████████████████████████████████████████████████████████| 91/91 [00:00<00:00, 283.51it/s]


In [231]:
khk['mevzuat_date_val']=pd.to_datetime(khk['mevzuat_date_val'],dayfirst=True)
khk['rega_date_val']=pd.to_datetime(khk['rega_date_val'],dayfirst=True)

In [232]:
print((khk['mevzuat_date_val']==khk['mevzuat_tarihi']).value_counts())
print((khk['rega_date_val']==khk['rega_tarihi']).value_counts())

True    91
dtype: int64
True    91
dtype: int64


### Resmi Gazete

In [233]:
#resmi_gazete['rega_date_val']=khk['data_text'].progress_apply(lambda x : find_dates(x,'rega','khk'))
#resmi_gazete['mevzuat_date_val']=khk['data_text'].progress_apply(lambda x : find_dates(x,'mevzuat','khk'))

### Genelge

In [248]:
unidecode.unidecode(genelge.data_text[990]).lower()

"veraset ve intikal vergisi kanunu ic genelgesi seri no: 1997/1\n\ntarih 17/01/1997\n\nsayi b.07.0.gel.0.60/6012-1430/2042\n\nkapsam \n\nt.c.\n\nmaliye bakanligi\n\ngelirler genel mudurlugu\n\nsayi : b.07.0.gel.0.60/6012-1430\n\nkonu :veraset ve intikal vergisi kanunu ic genelgesi\n\nseri no: 1997/1\n.......................valiligine (defterdarlik: gelir mudurlugu)\n\nbilindigi uzere, 7338 sayili veraset ve intikal vergisi kanununun 10 uncu maddesinin (b) bendinde, gayrimenkullerin ticari isletmeye dahil olsun veya olmasin emlak vergisine esas olan degerle degerlenecegi hukme baglanmis, ancak yapi kooperatifi hissesinin ne sekilde degerlenecegi hususunda bir hukme yer verilmemistir. vergi usul kanununun servetleri degerleme baslikli ucuncu bolumunde yer alan 291 inci maddesinde bir vergiye matrah olan servetin veya servet unsurlarinin degerlemesinde bu bolumde yazili esaslarin cari olacagi belirtilmekte, ayni kanunun 74 uncu maddesinde ise takdir komisyonlarinin gorevleri arasinda, yet

In [214]:
genelge.shape

(385, 16)

In [284]:
genelge[genelge.data_text.str.contains('tarih')].shape

(328, 16)

In [326]:
#for i in range(0,len(genelge)):
#    print(genelge.data_text[i].split("Tarih ",1)[1].split('\n')[0])

In [327]:
#genelge.data_text[5].split("Tarih ",1)[1].split('\n')[0]

In [328]:
#genelge['mevzuat_date_val']=khk['data_text'].progress_apply(lambda x : find_dates(x,'mevzuat','genelge'))

In [269]:
genelge.reset_index(drop=True,inplace=True)

In [280]:
genelge.data_text[15].split("Tarih ",1)[1].split('\n')[0]

'01/11/2000'